# Module

In [24]:
import warnings
warnings.filterwarnings(action='ignore')

import json
import os
import sys
import dotenv
import nest_asyncio
dotenv.load_dotenv()
import requests
import json
from glob import glob
import time
import pandas as pd
from tqdm import tqdm
from urllib.parse import unquote
from collections import defaultdict
from bs4 import BeautifulSoup

from pdf2image import convert_from_path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from PIL import Image, ImageDraw

# HTML 
* 하나의 PDF로 부터 나온 html tag는, id값이 1씩 increase 하도록 후처리
* 추후 해당 id를 활용해, json으로 부터 dounding_box와 page를 가져올 예정

In [45]:
# html 파일을 읽어서, id 값을 수정하는 함수
def change_html_tag_id(html_path, save_path):
    html = open(html_path).read()
    soup = BeautifulSoup(html, 'html.parser')
    re = 0
    for tag in soup.find_all():
        if tag.get('id'):
            tag['id'] = re
            re += 1
    with open(save_path, 'w') as f:    
        f.write(str(soup))  


In [51]:
html_paths = glob("../data/nursing/nursing_html/*")
for html_path in html_paths:
    file_name = os.path.basename(html_path)
    change_html_tag_id(html_path, f"../data/nursing/nursing_html2/{file_name}")

# JSON
* Document Parser로 부터 저장된 JSON 파일을 PDF 단위로 결합
* 결합시, id는 0부터 시작하여 HTML의 파일과 동일하게 매칭

In [2]:
def json_post_process(file_paths, save_folder_path):
    file_names = list(set([os.path.basename(file_path).split(".pdf")[0] for file_path in file_paths]))
    re = defaultdict(list)

    # 파일 이름이 같은 것들을 묶어준다.
    for file_name in file_names:
        for file_path in file_paths:
            if file_name in file_path:
                re[file_name].append(file_path)
    
    # 정렬 해 준다.
    for file_name in re:
        re[file_name] = sorted(re[file_name])
    
    for file_name in re:
        re_json = {"elements":[]}
        for file_path in re[file_name]:
            with open(file_path, "r") as f:
                data = json.load(f)
                re_json["elements"].extend(data["elements"])
        id_ = 0
        for r in re_json["elements"]:
            r["id"] = id_
            id_ += 1
            for b in r["bounding_box"]:
                b["x"] = b["x"]/1362
                b["y"] = b["y"]/1776
        
        with open(os.path.join(save_folder_path, file_name+".json"), 'w') as f:
            json.dump(re_json, f, indent=4)

In [3]:
json_post_process(glob("../data/nursing/nursing_downloads/*.json"), "../data/nursing/nursing_json")

# PDF to PNG
* PDF 파일을 PNG 이미지로 변환
* 형식 : 파일이름_page

In [13]:
pdf_paths = glob("../data/nursing/*.pdf")

In [14]:
pdf_paths

['../data/nursing/22장 신경계장애 대상자간호-(1).pdf',
 '../data/nursing/18장 요로계 장애 대상자 간호.pdf',
 '../data/nursing/14장 호흡기장애 대상자 간호-(2).pdf',
 '../data/nursing/17장 혈액계 장애 대상자 간호.pdf',
 '../data/nursing/21장 근골격계 장애.pdf',
 '../data/nursing/20장 유방장애 대상자 간호.pdf',
 '../data/nursing/27장 화상간호.pdf',
 '../data/nursing/25장 청각과 평형장애 대상자 간호.pdf',
 '../data/nursing/30 판권.pdf',
 '../data/nursing/26장 피부장애 대상자 간호.pdf',
 '../data/nursing/16장 혈관장애 대상자 간호.pdf',
 '../data/nursing/19장 남성 생식기계 대상자 간호.pdf',
 '../data/nursing/14장 호흡기장애 대상자 간호-(1).pdf',
 '../data/nursing/22장 신경계장애 대상자간호-(2).pdf',
 '../data/nursing/00 도입부(하권).pdf',
 '../data/nursing/23장 내분비장애 대상자 간호.pdf',
 '../data/nursing/24장 시각장애 대상자 간호.pdf',
 '../data/nursing/15장 심장장애 대상자 간호-(2).pdf',
 '../data/n

In [16]:
# for pdf_path in tqdm(pdf_paths):
#     images = convert_from_path(pdf_path, dpi=150)
#     file_name = os.path.basename(pdf_path).split(".pdf")[0]
    
#     if not os.path.exists(f"../data/nursing/nursing_images/{file_name}"):
#         os.makedirs(f"../data/nursing/nursing_images/{file_name}")

#     for i, image in enumerate(images):
#         image.save(f"../data/nursing/nursing_images/{file_name}/{i}.png")

100%|██████████| 19/19 [09:26<00:00, 29.82s/it]


# Bounding Box

In [28]:
title = "23장 내분비장애 대상자 간호"
t = """외분비샘은 관을 통하<br/>거나 직접 장관이나 피부에 효소(enzyme)나 분비물을<br/>내보내는 반면, 내분비샘은 호르몬을 직접 혈류로 분비<br/>하는 체계로서, 해부학적으로 연결된 것이 아니라 기능<br/>적으로 상호연결된 샘(gland)의 혼합구조이다.</p><br/><p data-category="paragraph" id="21" style="font-size:20px">내분비계는 샘과 샘조직, 표적기관 또는 수용체로 구</p><br/><p data-category="paragraph" id="22" style="font-size:20px">성된다. 주요 내분비샘은 시상하부(hypothalamus), 뇌<br/>하수체(pituitary gland), 갑상샘(thyroid gland), 부갑상<br/>샘 (parathyroid gland), 부신(adrenal gland), 췌장(이<br/>자)의 랑게르한스섬, 생식샘(고환과 난소) 등이다(그림<br/>23-1).</p><br/><p data-category="paragraph" id="23" style="font-size:20px">내분비샘에서 분비하는 호르몬은 표적기관에 작용하<br/>는 화학정보로서 기관의 기능을 자극하거나 억제한다.<br/>호르몬은 혈류를 따라 이동하여 표적세포의 수용체와</p><figure><img alt="시상 뇌하수체 (thalamus) (pituitary) 시상하부 (hypothalamus) 부갑상샘(갑상샘후면) (parathyroids) 갑상샘 시신경교차 (optic chiasph) (thyroid) 사용하는Trenjonrishopisis (nerve trunk) 선하수체 신경간 (adenohypophysis) 부신 누두돌기 (adrenals) (infundibular 열(cleft) 췌장(랑게르한스섬) process) (pancreas, islets of langerhans) 후엽 전엽 고환(남성) (posterior lobe) (anterior lobe) (testes) 난소(여성) (ovaries)" data-coord="top-left:(199,1018); bottom-right:(1200,1580)" id="24" style="font-size:16px"/></figure><p data-category="paragraph" id="25" style="font-size:16px">그림 23-1 내분비샘</p><header id="26" style="font-size:14px">CHAPTER 23 내분비장애 대상자 간호 I 549</header><p data-category="paragraph" id="27" style="font-size:18px">결합한다.</p><p data-category="paragraph" id="28" style="font-size:18px">내분비계와 신경계 작용을 통해 성장과 발달, 항상성 유<br/>지, 외부환경 변화의 응급요구에 대한 반응과 적응 및 세<br/>포복제가 일어난다."""

In [40]:
# html 형식 str 로 부터 tag id 추출
def extract_tag_id(html_str):
    soup = BeautifulSoup(html_str, 'html.parser')
    re = []
    for tag in soup.find_all():
        if tag.get('id'):
            re.append(int(tag.get('id')))
    
    if re[0] > 0:
        re.append(re[0]-1)
    return re

In [42]:
tag_ids = extract_tag_id(t)
tag_ids

[21, 22, 23, 24, 25, 26, 27, 28, 20]

In [43]:
with open(f"../data/nursing/nursing_json/{title}.json", "r") as f:
    json_data = json.load(f)

In [44]:
page_dict = defaultdict(list)
for tag_id in tag_ids:
    element = json_data["elements"][int(tag_id)]
    page_dict[element["page"]].append(element["bounding_box"])

In [45]:
page_dict

defaultdict(list,
            {3: [[{'x': 0.14023494860499267, 'y': 0.5180180180180181},
               {'x': 0.48751835535976507, 'y': 0.5180180180180181},
               {'x': 0.48751835535976507, 'y': 0.5349099099099099},
               {'x': 0.14023494860499267, 'y': 0.5349099099099099}],
              [{'x': 0.5124816446402349, 'y': 0.3778153153153153},
               {'x': 0.8715124816446402, 'y': 0.3778153153153153},
               {'x': 0.8715124816446402, 'y': 0.47297297297297297},
               {'x': 0.5124816446402349, 'y': 0.47297297297297297}],
              [{'x': 0.5139500734214391, 'y': 0.4774774774774775},
               {'x': 0.8707782672540382, 'y': 0.4774774774774775},
               {'x': 0.8707782672540382, 'y': 0.5349099099099099},
               {'x': 0.5139500734214391, 'y': 0.5349099099099099}],
              [{'x': 0.1461086637298091, 'y': 0.5731981981981982},
               {'x': 0.8810572687224669, 'y': 0.5731981981981982},
               {'x': 0.881057268

In [46]:
# png 불러와서 바운딩 박스 그리고 시각화
def draw_bounding_box(file_name, page_dict):
    result = []
    for page_ in page_dict: 
        image = Image.open(f"../data/nursing/nursing_images/{file_name}/{page_-1}.png")
        w, h = image.size
        draw = ImageDraw.Draw(image)
        for bbox in page_dict[page_]:
            x1 = bbox[0]["x"]*w
            y1 = bbox[0]["y"]*h
            x2 = bbox[1]["x"]*w
            y2 = bbox[2]["y"]*h
            draw.rectangle([x1, y1, x2, y2], outline=(255,0,0), width=2)
        image.save(f"./{file_name}_{page_-1}.png")
        print(f"SAVE - {file_name}_{page_-1}.png")

In [47]:
draw_bounding_box(title, page_dict)

SAVE - 23장 내분비장애 대상자 간호_2.png
SAVE - 23장 내분비장애 대상자 간호_3.png


In [48]:
image

NameError: name 'image' is not defined

In [1]:
from glob import glob
file_paths = glob("/home/livin/rag_pipeline/AutoRAG/data/nursing/nursing_html2/*")

In [5]:
file_paths = str(file_paths).replace("'", '"')

In [7]:
print(file_paths)

["/home/livin/rag_pipeline/AutoRAG/data/nursing/nursing_html2/25장 청각과 평형장애 대상자 간호.html", "/home/livin/rag_pipeline/AutoRAG/data/nursing/nursing_html2/22장 신경계장애 대상자간호-(1).html", "/home/livin/rag_pipeline/AutoRAG/data/nursing/nursing_html2/14장 호흡기장애 대상자 간호-(2).html", "/home/livin/rag_pipeline/AutoRAG/data/nursing/nursing_html2/24장 시각장애 대상자 간호.html", "/home/livin/rag_pipeline/AutoRAG/data/nursing/nursing_html2/30 판권.html", "/home/livin/rag_pipeline/AutoRAG/data/nursing/nursing_html2/15장 심장장애 대상자 간호-(2).html", "/home/livin/rag_pipeline/AutoRAG/data/nursing/nursing_html2/18장 요로계 장애 대상자 간호.html", "/home/livin/rag_pipeline/AutoRAG/data/nursing/nursing_html2/00 도입부(하권).html", "/home/livin/rag_pipeline/AutoRAG/data/nursing/nursing_html2/21장 근골격계 장애.html", "/home/livin/rag_pipeline/AutoRAG/data/nursing/nursing_html2/16장 혈관장애 대상자 간호.html", "/home/livin/rag_pip